# Uso de selectores

Si conocéis el lenguaje CSS entonces ya sabéis lo que vamos a aprender en esta lección.

Los selectores son una técnica para especificar de forma más concreta qué elementos recuperar del árbol:

| Selector                 | Descripción                      | 
|--------------------------|----------------------------------| 
| Etiquetas                | `soup.select("tag")`             | 
| Identificadores          | `soup.select("#id")`             | 
| Clases                   | `soup.select(".class")`          | 
| Atributos                | `soup.select("tag[attr)`         |  
| Etiquetas en etiquetas   | `soup.select("parent child")`    | 

Vamos a scrapear [la página de Python](https://es.wikipedia.org/wiki/Python) en la Wikipedia para poner en práctica algunos de estos selectores:

In [ ]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://web.archive.org/web/20220722211457/https://es.wikipedia.org/wiki/Python")
soup = BeautifulSoup(req.text)

title = soup.select("title")[0].getText()
print(title)

Haciendo **clic derecho inspeccionar en cualquier navegador** podemos ver el código fuente mientras seleccionamos los elementos de la página.

Por ejemplo, en el primer parágrafo del documento encontramos un resumen del artículo:

In [ ]:
resumen = soup.select("p")[0].getText()
print(resumen)

## Scrapeando el índice

La Wikipedia tiene lo que se conoce como **Tabla de contenidos**, un índice del documento. 

Según el inspector su etiqueta abre con este tag:

```html
<div id="toc" class="toc" role="navigation" aria-labelledby="mw-toc-heading">
```

Es una capa `div` pero tiene un identificador `id` así que podemos usar un selector y luego podemos extraer sus enlaces `a`:

In [ ]:
toc = soup.select("#toc")[0]

for a in toc.select("a"):
    print(a.getText())

Con un poco de ingenio y una expresión regular podemos mostrar únicamente los enlaces de primer nivel:

In [ ]:
import re

for a in toc.select("a"):
    text = a.getText()
    if re.match(r"\d+ ", text):
        print(text)

O formatear los de segundo y tercer nivel con espacios:

In [ ]:
import re

for a in toc.select("a"):
    text = a.getText()
    if re.match(r"\d+ ", text):
        print(text)
    elif re.match(r"\d+.\d+ ", text):
        print(" ", text)
    elif re.match(r"\d+.\d+.\d+ ", text):
        print("   ", text)

Como véis es cuestión de ser creativo y utilizar las herramientas de las que disponemos.

## Scrapeando la caja de información

Otro elemento interesante que encontramos en el artículo es la caja de información arriba a la derecha, si la insepeccionamos veremos que es una tabla con la clase `infobox`: 


```html
<table class="infobox" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;">
```

En el lenguaje HTML las tablas se forman a partir de etiquetas con filas `tr` que contienen cabeceras `th` o celdas con datos `td`.

Podemos empezar recorriendo las filas a ver qué encontramos:

In [ ]:
tr_tags = soup.select(".infobox tr")

for tr_tag in tr_tags:
    print(tr_tag.getText())

No está mal pero podríamos reestructurar el contenido de forma que sea más legible.

Como cada `tr` tiene en teoría dos columnas, una con la cabecera `th` a la izquierda y el texto `td` a la derecha, podemos usar los índices para formatear el texto de salida:

In [ ]:
tr_tags = soup.select(".infobox tr")

for tr_tag in tr_tags:
    th_tags = tr_tag.select("th")
    td_tags = tr_tag.select("td")
    if len(th_tags) > 0 and len(td_tags) > 0:
        print(f"{th_tags[0].getText().strip()}: {td_tags[0].getText().strip()}")

## Scrapeando una imagen

Por último nos quedó pendiente el logo del `infobox`, si inspeccionamos la imagen veremos que tiene un tag `img` con muchos atributos:

```html
<img alt="Python-logo-notext.svg" src="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png" decoding="async" width="100" height="100" srcset="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/150px-Python-logo-notext.svg.png 1.5x, //web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png 2x" data-file-width="110" data-file-height="110">
```

Es un poco difícil recuperar este elemento porque no tiene un identificador único, pero se me ocurre algo mejor...

¿No se encuentra la imagen principal siempre en la tabla `infobox`? Pues vamos a utilizarla para recuperar la primera imagen en su interior:

In [ ]:
img = soup.select(".infobox img")[0]

print(img)

Genial, solo tenemos que recuperar el enlace de la imagen en su atributo src:

In [ ]:
print(img['src'])

Utilizando el propio módulo `requests` podemos guardar la imagen en el directorio de este mismo notebook con el nombre que queramos:

In [ ]:
# Ponemos el protocolo https: delante porque el enlace no lo incluye
response = requests.get(f"https:{img['src']}")

if response.status_code == 200:
    with open("image.png", 'wb') as f:
        f.write(response.content)

Si no falla en principio es que se ha descargado, podemos mostrar la imagen en markdown con un simple código:

```markdown
![](image.png)
```

![](image.png)

Os recomiendo experimentar por vuestra cuenta para aprender más, en la [documentación oficial](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) tenéis toda la información sobre `bs4` y el manejo de la jerarquía, también tenéis [más ejemplos sobre el uso de select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors).